In [ ]:
!pip install git+https://github.com/openai/whisper.git edge_tts bitsandbytes accelerate

In [1]:
import whisper
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import bitsandbytes as bnb
import edge_tts
import asyncio
from IPython.display import Audio, display

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class VoiceAssistant(nn.Module):
  def __init__(self, assistant_name: str, device, audio_transcriber='base.en', vad_threshold=0.5, text_processor='mistralai/Mistral-7B-Instruct-v0.3', restriction=None, voice='female'):
    super(VoiceAssistant, self).__init__()

    self.assistant_name = assistant_name

    self.audio_transcriber = whisper.load_model(audio_transcriber)


    tokenizer = AutoTokenizer.from_pretrained(text_processor)

    model = AutoModelForCausalLM.from_pretrained(
        text_processor,
        load_in_4bit=True,
        device_map="auto",
        torch_dtype=torch.float16
    )

    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

    self.text_processor = pipeline("text-generation", model=text_processor, device=device)


    self.voice = 'en-US-ChristopherNeural' if voice.lower() == 'male' else 'en-US-JennyNeural'
    self.rate = 0
    self.volume = 0
    self.pitch = 0

    self.restriction = restriction

    self.context = [
      {"role": "system", "content": f'You are a personal assistant chatbot, your name is {assistant_name}'},
    ]

  def reset_context(self):
    self.context = [
      {"role": "system", "content": f'You are a personal assistant chatbot, your name is {self.assistant_name}'},
    ]

  def change_voice_settings(self, voice, speed=0, volume=0, pitch=0):
    self.voice = 'en-US-ChristopherNeural' if voice.lower() == 'male' else 'en-US-JennyNeural'
    self.rate += speed
    self.volume += volume
    self.pitch += pitch

  def set_restriction(self, restriction = None):
    self.restriction = restriction

  def forward(self, x):

    x = self.audio_transcriber.transcribe(x)['text']
    assistant.context.append({"role": "user", "content": x})

    x = assistant.text_processor(assistant.context, max_new_tokens=1024, num_return_sequences=1)
    x = x[0]['generated_text']
    assistant.context=x
    x = x[-1]['content']

    if self.restriction is not None and self.restriction != -1:
      val = -1
      q_val = -1
      d_val = -1
      for i in range(self.restriction):
        d_val = x.find('.', d_val + 1)
        q_val = x.find('?', q_val + 1)
        val = min(d_val, q_val)
        d_val = q_val = val

      x = x[-1]['content'][:val+1]

    communicate = edge_tts.Communicate(x, self.voice, rate=f'+{self.rate}%', volume=f'+{self.volume}%', pitch=f'+{self.pitch}Hz')
    communicate.save_sync(f'/content/output_{len(self.context)//2}.mp3')
    display(Audio(f'/content/output_{len(self.context)//2}.mp3'))

    return x

In [4]:
settings = {
    'assistant_name': 'Sara',
    'audio_transcriber':'base.en',
    'device': device,
    'vad_threshold':0.5,
    'text_processor':'HuggingFaceTB/SmolLM-1.7B-Instruct',
    'restriction':None,
    'voice':'female'
}

In [5]:
assistant = VoiceAssistant(**settings).to(device)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [6]:
display(Audio('/content/hello.mp4'))

In [7]:
assistant('/content/hello.mp4')

"My name is Sara. I'm a personal assistant chatbot."

In [8]:
speed = 10
volume = 10
pitch = 50

In [9]:
assistant.change_voice_settings('male', speed, volume, pitch)

In [10]:
display(Audio('/content/book.mp4'))

In [11]:
assistant('/content/book.mp4')

'One of my favorite science fiction books is "Dune" by Frank Herbert. It\'s a classic and thought-provoking novel that explores complex themes and ideas.'

In [12]:
assistant.change_voice_settings('female', -speed, -volume, -pitch)

In [13]:
display(Audio('/content/vacation.mp4'))

In [14]:
assistant('/content/vacation.mp4')

"Here are three popular tourist attractions in Paris:\n\n1. **The Eiffel Tower**: An iconic iron lattice tower built for the 1889 World's Fair, offering stunning views of the city from its top.\n2. **The Louvre Museum**: One of the world's largest and most famous museums, housing an impressive collection of art and artifacts from around the world, including the Mona Lisa.\n3. **The Arc de Triomphe**: A triumphal arch honoring the soldiers who fought and died for France, with stunning views of the city from its top.\n\nI hope you find these suggestions helpful!"

In [15]:
for c in assistant.context:
  print(c)
  print()

{'role': 'system', 'content': 'You are a personal assistant chatbot, your name is Sara'}

{'role': 'user', 'content': ' Hello there, what is your name?'}

{'role': 'assistant', 'content': "My name is Sara. I'm a personal assistant chatbot."}

{'role': 'user', 'content': ' Recommend a good book in science fiction.'}

{'role': 'assistant', 'content': 'One of my favorite science fiction books is "Dune" by Frank Herbert. It\'s a classic and thought-provoking novel that explores complex themes and ideas.'}

{'role': 'user', 'content': ' I am planning a trip to Paris, suggest the names of three tourist attractions.'}

{'role': 'assistant', 'content': "Here are three popular tourist attractions in Paris:\n\n1. **The Eiffel Tower**: An iconic iron lattice tower built for the 1889 World's Fair, offering stunning views of the city from its top.\n2. **The Louvre Museum**: One of the world's largest and most famous museums, housing an impressive collection of art and artifacts from around the worl